In [1]:
import pandas as pd
import os
import time
from main_code.analysis import Analysis
from main_code.read_data import Read_data
import warnings
warnings.filterwarnings("ignore")
"""for debug"""
%load_ext autoreload
%autoreload 2
os.environ["OMP_NUM_THREADS"] = "30"

from config.path import Path
file_path = Path

### main script for calculate feature value

In [2]:
def cal_all_ic(return_time=0.25,factor_back_time=2, ic_types="normal_ic", \
    factor_type="D",factor_derive=True):#'rank_ic'or normal_ic,return_time=0.25
    """calculate factor, return and stored for further analysis"""
    
    start_time = time.time()
    raw_data_obj = Read_data()
    
    os.chdir(file_path+'/2022年T合约tick数据')
    fname_list = [x for x in os.listdir() if '20220701' in x or '20220701' in x]

    for fname in fname_list:
        
        df_oneday = raw_data_obj.read_future_data(fname)
        ana = Analysis(data=df_oneday,data_underlying='future',data_level=5,\
            factor_derive=factor_derive, factor_back_time=factor_back_time)
        return_df = ana.cal_return(mins=return_time)
        
        factor_df = ana.cal_feature(factor_type)
        print(factor_df)
        
        factor_df = factor_df.loc[factor_df['TradingTime'].isin(return_df['TradingTime'])].reset_index(drop=True)
        return_df = return_df.loc[return_df['TradingTime'].isin(factor_df['TradingTime'])].reset_index(drop=True)
        factor_return_df = pd.concat([factor_df, return_df['return']],axis=1)#, axis=1, ignore_index=True)

        time_info =fname.split('.')[0]
        h5_file_name = 'type_%s_factor_and_return_%s_min_back_%s_min_%s_%s'\
            %(factor_type, str(return_time), factor_back_time, ic_types,time_info)
        factor_return_file_path = file_path+'/data/ic_results_data/%s_factor_data/'%factor_type
        if not os.path.exists(factor_return_file_path):
            os.makedirs(factor_return_file_path)
        factor_return_df.to_hdf(factor_return_file_path+h5_file_name+".h5", index=False,key="results")

    end_time = time.time()
    print("total time is %0.2f s"%(end_time-start_time))
    if return_time == None:
        return_time = "one_tick"
    
    end_time2 = time.time()
    print("total time (with store time) is %0.2f s"%(end_time2-start_time))


input_para=[1, 1, 'normal_ic', 'B', False]#
cal_all_ic(return_time=input_para[0],factor_back_time=input_para[1], ic_types=input_para[2], \
    factor_type=input_para[3],factor_derive=input_para[4])#'rank_ic'or normal_ic

(30601, 20)
      Symbol             TradingTime  snapshot_direct_imbalance_vol1  \
1      T2209 2022-07-01 09:30:00.800                           -50.0   
2      T2209 2022-07-01 09:30:01.300                           148.0   
3      T2209 2022-07-01 09:30:01.800                            54.0   
4      T2209 2022-07-01 09:30:02.300                           -69.0   
5      T2209 2022-07-01 09:30:02.800                            42.0   
...      ...                     ...                             ...   
30597  T2209 2022-07-01 15:14:58.300                            10.0   
30598  T2209 2022-07-01 15:14:58.800                           -20.0   
30599  T2209 2022-07-01 15:14:59.300                             7.0   
30600  T2209 2022-07-01 15:14:59.800                            -1.0   
30601  T2209 2022-07-01 15:15:00.300                             0.0   

       snapshot_direct_imbalance_vol2  increment_imbalance_volume1  \
1                                -1.0                

PermissionError: file ``/Volumes/EXTERNAL_USB/swhy_data/data/ic_results_data/B_factor_data/type_B_factor_and_return_1_min_back_1_min_normal_ic_20220701.h5`` exists but it can not be written

In [30]:
cal_all_ic(return_time=input_para[0],factor_back_time=input_para[1], ic_types=input_para[2], \
    factor_type=input_para[3],factor_derive=input_para[4])#'rank_ic'or normal_ic

total time is 0.10 s
total time (with store time) is 0.10 s
